In [2]:
#З якоїсь причини у цьому коді при спробі побудувати графік ядро Юпітера помирає та код не виконується
#У попередній лабораторній subplots будувалися без помилок, рішення з sttackoverflow не допомогли

#Колаб видає помилку у блоці коду №8(тренування мережі):
#Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as plt
from tensorflow.keras.datasets import mnist

In [3]:
num_classes = 10
num_features = 784
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100
n_hidden_1 = 128
n_hidden_2 = 256

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name="w"
        )
        self.b = tf.Variable(tf.zeros([out_features]), name="b")

    def __call__(self, x, activation=0):
        y = tf.matmul(x, self.w) + self.b
        if activation != 0:
            return tf.nn.softmax(y)
        else:
            return tf.nn.sigmoid(y)


class NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.layer_1 = DenseLayer(in_features=num_features, out_features=n_hidden_1)
    self.layer_2 = DenseLayer(in_features=n_hidden_1, out_features=n_hidden_2)
    self.layer_3 = DenseLayer(in_features=n_hidden_2, out_features=num_classes)

  def __call__(self, x):
    x1 = self.layer_1(x, activation=0)
    x2 = self.layer_2(x1, activation=0)
    x3 = self.layer_3(x2, activation=1)
    return x3

In [6]:
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

def accuracy(y_pred, y_true):
    acc = tf.metrics.Accuracy()
    acc.update_state(y_true, tf.argmax(y_pred, axis=1))
    return acc.result()

In [7]:
neural_net = NN(name="mnist")

def train(nn, input_x, output_y):
  optimizer = tf.optimizers.SGD(learning_rate)
  with tf.GradientTape() as g:
    pred = neural_net(input_x)
    loss = cross_entropy(pred, output_y)
    
    trainable_variables = nn.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [8]:
loss_history = []
accuracy_history = []

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps)):
    train(neural_net, batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x)
        current_loss = cross_entropy(pred, batch_y)
        loss_history.append(current_loss)
        
        current_accuracy = accuracy(pred, batch_y)
        accuracy_history.append(current_accuracy)
        print(f"Step: {step}, Loss: {current_loss}, Accuracy: {current_accuracy}")

Step: 0, Loss: 2414.99169921875, Accuracy: 0.08203125
Step: 100, Loss: 359.6050109863281, Accuracy: 0.66015625
Step: 200, Loss: 161.2314453125, Accuracy: 0.80859375
Step: 300, Loss: 177.51800537109375, Accuracy: 0.79296875
Step: 400, Loss: 228.4430389404297, Accuracy: 0.796875
Step: 500, Loss: 150.48651123046875, Accuracy: 0.8125
Step: 600, Loss: 110.1222152709961, Accuracy: 0.859375
Step: 700, Loss: 137.86660766601562, Accuracy: 0.84375
Step: 800, Loss: 105.77922058105469, Accuracy: 0.8984375
Step: 900, Loss: 117.264892578125, Accuracy: 0.859375
Step: 1000, Loss: 91.23553466796875, Accuracy: 0.8984375
Step: 1100, Loss: 85.75437927246094, Accuracy: 0.90625
Step: 1200, Loss: 88.54682159423828, Accuracy: 0.890625
Step: 1300, Loss: 99.02114868164062, Accuracy: 0.88671875
Step: 1400, Loss: 75.54603576660156, Accuracy: 0.9453125
Step: 1500, Loss: 77.38594818115234, Accuracy: 0.9296875
Step: 1600, Loss: 83.52347564697266, Accuracy: 0.90234375
Step: 1700, Loss: 65.01136779785156, Accuracy: 0.

In [4]:
#Смерть ядра
"""
fig, axs = plt.subplots(figsize=(16, 10))
axs.plot(loss_history, 'r', label='loss', linestyle=":")
axs.legend()

axs2=axs.twinx()
axs2.plot(accuracy_history, 'b', label='accuracy', linestyle=":")
plt.legend()

plt.title('Залежності зміни точності та втрат від кроку', fontsize=20, color='g')
plt.show()
"""

'\nfig, axs = plt.subplots(figsize=(16, 10))\naxs.plot(loss_history, \'r\', label=\'loss\', linestyle=":")\naxs.legend()\n\naxs2=axs.twinx()\naxs2.plot(accuracy_history, \'b\', label=\'accuracy\', linestyle=":")\nplt.legend()\n\nplt.title(\'Залежності зміни точності та втрат від кроку\', fontsize=20, color=\'g\')\nplt.show()\n'

In [9]:
neural_net_accuracy = accuracy(neural_net(x_train), y_train)
print(f"Accuracy: {neural_net_accuracy}")

Accuracy: 0.9200833439826965


In [ ]:
test_img = np.random.permutation(x_test.shape[0])[:10]
y_test_true = y_test[test_img]
pred_data = [np.argmax(x) for x in neural_net(x_test[test_img])]

#Смерть ядра
"""
plt.figure(figsize=(16, 6))
for i in range(10):
    print(f"True: {y_test_true[i]} Predict: {pred_data[i]} {True if y_test_true[i] - pred_data[i] == 0 else False}")
    plt.subplot(1, 10, (i + 1))
    plt.imshow(x_test[test_img[i]].reshape(28, 28), cmap='gray')
    plt.text(x=10, y=-10, s=pred_data[i], fontsize=18)
    
plt.tight_layout()
plt.show()
"""

True: 3 Predict: 7 False
